# Backpropagation

In the lecture we have already discussed the gradient backpropagation algorithm for calculating derivatives in neural networks. In this exercise you will implement this algorithm in a similar fashion as it is done in PyTorch.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import nn
import toolbox as tb
from toolbox import Tensor

To illustrate the general idea, run the following lines of code.

In [2]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

In [3]:
d.backward()

In [4]:
print(d.grad)
print(c.grad)
print(b.grad)
print(a.grad)

1.0
1.0
1.0
2.0


Let's try to understand what's going on here. After we have run ``d.backward()`` the variables ``a``, ``b`` and ``c`` involved in the computation of ``d`` now feature an attribute ``grad``. This attribute represents the gradient of each variable w.r.t. computing ``d``:
$$
\frac{\mathrm d \mathop{}\! d}{\mathrm d \mathop{}\! c} = \frac{\mathrm d \mathop{}\! c}{\mathrm d \mathop{}\! c} + \frac{\mathrm d \mathop{}\! a}{\mathrm d \mathop{}\! c} = 1 + 0 = 1, \\
\frac{\mathrm d \mathop{}\! d}{\mathrm d \mathop{}\! b} = \frac{\mathrm d \mathop{}\! c}{\mathrm d \mathop{}\! b} + \frac{\mathrm d \mathop{}\! a}{\mathrm d \mathop{}\! b} = 1 + 0 = 1, \\
\frac{\mathrm d \mathop{}\! d}{\mathrm d \mathop{}\! a} = \frac{\mathrm d \mathop{}\! c}{\mathrm d \mathop{}\! a} + \frac{\mathrm d \mathop{}\! a}{\mathrm d \mathop{}\! a} = \left(\frac{\mathrm d \mathop{}\! a}{\mathrm d \mathop{}\! a} + \frac{\mathrm d \mathop{}\! b}{\mathrm d \mathop{}\! a}\right) + \frac{\mathrm d \mathop{}\! a}{\mathrm d \mathop{}\! a} = \left( 1 + 0 \right) + 1 = 2
$$

But how does ``a`` know about the computations going on after its creation? The answer is that every operation we apply to tensors creates a new object representing this operation with all its variables involved. The output tensor of these operations saves a reference to the the operation which created it in the attribute ``grad_fn``:

In [5]:
c = a + b
d = c + a

print(d.grad_fn)
print(c.grad_fn)
print(b.grad_fn)
print(a.grad_fn)

None
None


From the output above you can see that ``c`` and ``d`` resulted from addition operations. Variables ``a`` and ``b`` were created from scratch and therfore have the value ``None`` as ``grad_fn``. This mechnanism implicitly builds a computation graph with all the dependencies among the variables. The function call ``d.backward()`` is a shorthand for ``d.backward(1.)`` and starts the gradient backpropagation involving all variables involved in the creation of ``d``. The only exception are tensors for which the ``requires_grad`` evaluates to ``False``, which is also the default value for the creation of a tensor object.

Let's try to call the ``backward`` function two times in a row:

In [6]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

d.backward()
d.backward()

AttributeError: 'Tensor' object has no attribute 'grad_fn'

The error is thrown, because the computation graph is deleted after the first gradient backpropagation as most of the times there is no use in keeping this graph alive. Therefore a new computation graph is built for every forward pass.

Also observe the behaviour of the gradient calculation below:

In [7]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

d.backward()

c = a + b
d = c + a

d.backward()

print(d.grad)
print(c.grad)
print(b.grad)
print(a.grad)

1.0
1.0
2.0
4.0


The gradients are accumulated over time and hence the values above are incorrect. Thus it will be important to reset the gradient values for parameters we want to optimize over to zero after each forward pass:

In [8]:
a = Tensor(1., requires_grad=True)
b = Tensor(2., requires_grad=True)

c = a + b
d = c + a

d.backward()
a.zero_grad()
b.zero_grad()

c = a + b
d = c + a

d.backward()

print(d.grad)
print(c.grad)
print(b.grad)
print(a.grad)

1.0
1.0
1.0
2.0


Have a look at the file ``toolbox.py`` and make sure you understand the basic mechanism of how it implements the creation of computation graphs. Then follow the comments and implement the backward pass for ``Mul``, ``Div``, ``Pow``, ``MatMul``, ``ReLU``, ``Exp``, ``Log``, ``Sum``, ``Mean`` and ``GetItem``. The addition function ``Add`` is already implemented and may serve as a reference. Test you implementations by running the cells below:

## Mul [1 point]

In [9]:
a = Tensor(2., requires_grad=True)
b = Tensor(3., requires_grad=True)

c = a * b
c.backward(2.)

assert np.abs(a.grad-6.)<1e-6 and np.abs(b.grad-4.)<1e-6, "Multiplication doesn't work properly!"

## Div [1 point]

In [10]:
a = Tensor(np.array([1., 2.]), requires_grad=True)
b = Tensor(np.array([3., 4.]), requires_grad=True)

c = a / b
c.backward(0.5)

assert np.linalg.norm(a.grad-np.array([1./6, 1./8]))<1e-6 and \
np.linalg.norm(b.grad-np.array([-1./18, -1./16]))<1e-6, \
"Division doesn't work properly!"

## Pow [1 point]

In [11]:
A = Tensor(np.array([[1., 2.], [3., 4.]]), requires_grad=True)

b = A ** 4
b.backward(np.array([[-1., 0], [1., 2.]]))

assert np.linalg.norm(A.grad-np.array([[-4., 0.], [108., 512.]]))<1e-6, "Exponentiation doesn't work properly!"

## MatMul [1 point]

In [12]:
A = Tensor(np.array([[1., 2.], [3., 4.]]), requires_grad=True)
b = Tensor(np.array([[1.], [2.]]), requires_grad=True)
c = Tensor(np.array([[3., 4.]]), requires_grad=True)

d = c @ A @ b
d.backward(-1.5)

assert np.linalg.norm(A.grad-np.array([[-4.5, -9.], [-6., -12.]]))<1e-6 and \
np.linalg.norm(b.grad-np.array([[-22.5], [-33.]]))<1e-6 and \
np.linalg.norm(c.grad-np.array([[-7.5, -16.5]]))<1e-6, "Matrix multiplication doesn't work properly!"

## ReLU [1 point]

In [14]:
A = Tensor(np.array([[-1., -2,], [0.5, 2.], [1., 5.]]), requires_grad=True)

b = tb.relu(A)
b.backward(np.array([[4., 3.], [2., 1.], [0., -1.]]))

assert np.linalg.norm(A.grad-np.array([[0., 0.], [2., 1.], [0., -1.]]))<1e-6, "ReLU doesn't work properly!"

AssertionError: ReLU doesn't work properly!

## Exp [1 point]

In [15]:
A = Tensor(np.array([[-1.], [2.]]), requires_grad=True)

b = tb.exp(A)
b.backward(np.array([[-1.], [2.]]))

assert np.linalg.norm(A.grad-np.exp(A.data)*np.array([[-1.], [2.]]))<1e-6, \
"The exponential function doesn't work properly!"

## Log [1 point]

In [16]:
A = Tensor(np.array([0.1, 1., 2., np.exp(100.)]), requires_grad=True)

b = tb.log(A)
b.backward(1.5)

assert np.linalg.norm(A.grad-np.array([15., 1.5, 0.75, 1.5*np.exp(-100.)]))<1e-6, "Logarithm doesn't work properly!"

## Sum [1 point]

In [17]:
A = Tensor(np.array([[1., 2.], [3., 4.]]), requires_grad=True)

b = A.sum(dim=1)
b = b.sum()
b.backward(2.5)

assert np.linalg.norm(A.grad-2.5*np.ones((2, 2)))<1e-6, "Summation doesn't work properly!"

AssertionError: Summation doesn't work properly!

## Mean [1 point]

In [22]:
A = Tensor(np.array([[1., 2.], [3., 4.]]), requires_grad=True)

b = A.mean()
b.backward()

assert np.linalg.norm(A.grad-0.25*np.ones((2, 2)))<1e-6, "Mean doesn't work properly!"

## GetItem [1 point]

In [23]:
A = Tensor(np.array([[1., 2.], [3., 4.]]), requires_grad=True)

b = A[0, 1]
b.backward(5.)

assert np.linalg.norm(A.grad-5.*np.array([[0., 1.], [0., 0.]]))<1e-6, "Indexing doesn't work properly!"

If the above tests were successful, you should be able to backpropagate through a network:

In [24]:
network = nn.Network()
network.add_layer(nn.LinearLayer(np.array([[1., 2.], [3., 4.]]), \
                                np.array([[1.], [2.]])))
network.add_layer(nn.ReLU())
network.add_layer(nn.LinearLayer(np.array([[1., 2.], [3., 4.], [5., 6.]]), \
                 np.array([[1.], [2.], [3.]])))

loss = nn.CrossEntropyLoss()

data = Tensor(np.array([[1.], [2.]]))
target = 1

l = loss.forward(network.forward(data), target)
l.backward()

assert (np.linalg.norm(network.layers[0].W.grad-np.array([[2., 4.], [2., 4.]]))<1e-6 and
np.linalg.norm(network.layers[0].b.grad-np.array([[2.], [2.]]))<1e-6 and
np.linalg.norm(network.layers[2].W.grad-np.array([[ 8.00168889e-34,  1.73369926e-33],
                                                  [-6.00000000e+00, -1.30000000e+01],
                                                  [ 6.00000000e+00,  1.30000000e+01]]))<1e-6 and
np.linalg.norm(network.layers[2].b.grad-np.array([[ 1.33361482e-34],
                                                  [-1.00000000e+00],
                                                  [ 1.00000000e+00]]))<1e-6), "Something is wrong..."

AssertionError: Something is wrong...